In [243]:
import os
from glob import glob
import subprocess
import re
from collections import defaultdict, OrderedDict

TPTP_DIR = os.path.expanduser('~/TPTP-v7.0.0')
THEORY_PROBLEMS = [path for path in glob(TPTP_DIR + '/**/*.p', recursive=True) if '=' in path]
PROOF_DIR = os.path.join(os.getcwd(), 'proofs')
VAMPIRE_TIMEOUT = '0.25'
LINE_PATTERN = re.compile(r'^(\d+). (.+) \[([^0-9]*)([0-9,]*)\]$')
VAR_PATTERN = re.compile(r'(X\d+)([^0-9]|$)')

def vampire(path):
    result = subprocess.run(
        ['vampire', '--time_limit', VAMPIRE_TIMEOUT, path],
        cwd=TPTP_DIR,
        stdout=subprocess.PIPE
    )
    return result.stdout.decode('ascii') if result.returncode == 0 else None

def proof_path(problem):
    return os.path.join(PROOF_DIR, os.path.splitext(os.path.basename(problem))[0] + '.proof')

def write_proof(problem, proof):
    filename = proof_path(problem)
    with open(filename, 'w') as file:
        file.write(proof)

def proof_paths():
    return glob(PROOF_DIR + '/*.proof')

def clean_formula(formula):
    formula = formula.replace("(TD)", "").strip()
    var_map = {'next': 0}

    def sub_var(match):
        digits, rest = match.groups()
        n = int(digits[1:])
        if n not in var_map:
            var_map[n] = var_map['next']
            var_map['next'] += 1
        return "Y{}{}".format(var_map[n], rest)

    return VAR_PATTERN.subn(sub_var, formula)[0]

def parse_proof(proof):
    result = OrderedDict()

    for line in proof.split('\n'):
        match = LINE_PATTERN.match(line)
        if match:
            index, formula, inference, parents = match.groups()
            parents = [int(parent) for parent in parents.split(',')] if parents else []
            result[int(index)] = (clean_formula(formula), inference.strip(), parents)
    return result

def theory_results(proof):
    axioms = {}
    lemmas = {}
    for key, (formula, inference, parents) in proof.items():
        if inference == 'theory axiom':
            axioms[key] = formula
        elif parents and all(parent in axioms or parent in lemmas for parent in parents):
            lemmas[key] = formula
    return (set(axioms.values()), set(lemmas.values()))

In [189]:
if not os.path.exists(PROOF_DIR):
    os.mkdir(PROOF_DIR)

for i, problem in enumerate(THEORY_PROBLEMS):
    proof = vampire(problem)
    print('[{}]\t{}\t{}/{}'.format('✓' if proof else '✘', os.path.basename(problem), i, len(THEORY_PROBLEMS)))
    if proof:
        write_proof(problem, proof)

[✓]	MSC022=2.p	0/1364
[✘]	MSC028=1.p	1/1364
[✘]	MSC023=2.p	2/1364
[✘]	DAT101=1.p	3/1364
[✓]	DAT020=1.p	4/1364
[✓]	DAT081=1.p	5/1364
[✘]	DAT105=1.p	6/1364
[✓]	DAT028=1.p	7/1364
[✓]	DAT098=1.p	8/1364
[✓]	DAT034=1.p	9/1364
[✘]	DAT109=1.p	10/1364
[✓]	DAT030=1.p	11/1364
[✓]	DAT019=1.p	12/1364
[✓]	DAT073=1.p	13/1364
[✘]	DAT095=1.p	14/1364
[✘]	DAT111=1.p	15/1364
[✘]	DAT086=1.p	16/1364
[✓]	DAT084=1.p	17/1364
[✘]	DAT080=1.p	18/1364
[✓]	DAT006=1.p	19/1364
[✘]	DAT050=1.p	20/1364
[✓]	DAT002=1.p	21/1364
[✓]	DAT017=1.p	22/1364
[✘]	DAT051=1.p	23/1364
[✓]	DAT091=1.p	24/1364
[✘]	DAT048=1.p	25/1364
[✓]	DAT009=1.p	26/1364
[✓]	DAT088=1.p	27/1364
[✘]	DAT008=1.p	28/1364
[✓]	DAT085=1.p	29/1364
[✓]	DAT003=1.p	30/1364
[✓]	DAT083=1.p	31/1364
[✘]	DAT015=1.p	32/1364
[✓]	DAT041=1.p	33/1364
[✓]	DAT045=1.p	34/1364
[✓]	DAT077=1.p	35/1364
[✘]	DAT053=1.p	36/1364
[✓]	DAT069=1.p	37/1364
[✘]	DAT023=1.p	38/1364
[✓]	DAT062=1.p	39/1364
[✓]	DAT035=1.p	40/1364
[✘]	DAT038=1.p	41/1364
[✓]	DAT066=1.p	42/1364
[✘]	DAT012=1.p	43/136

In [231]:
print("{}% of problems were solved.".format(100 * len(proof_paths()) / len(THEORY_PROBLEMS)))

49.193548387096776% of problems were solved.


In [244]:
axiom_counts = defaultdict(lambda: 0)
lemma_counts = defaultdict(lambda: 0)

for path in proof_paths():
    with open(path, 'r') as f:
        axioms, lemmas = theory_results(parse_proof(f.read()))
        for axiom in axioms:
            axiom_counts[axiom] += 1
        for lemma in lemmas:
            lemma_counts[lemma] += 1

print("Count\tAxiom\n")
for axiom, count in sorted(axiom_counts.items(), key=lambda x: x[1], reverse=True):
    print("{}\t{}".format(count, axiom))

print("\n\n---------------------\n\n")

print("Count\tLemma\n")
for lemma, count in sorted(lemma_counts.items(), key=lambda x: x[1], reverse=True):
    print("{}\t{}".format(count, lemma))

Count	Axiom

113	~$less(Y0,Y0)
84	$sum(Y0,Y1) = $sum(Y1,Y0)
50	$less(Y0,$sum(Y1,1)) | $less(Y1,Y0)
47	~$less(Y0,Y1) | ~$less(Y2,Y0) | $less(Y2,Y1)
44	$less(Y0,Y1) | $less(Y1,Y0) | Y1 = Y0
44	$sum(Y0,$sum(Y1,Y2)) = $sum($sum(Y0,Y1),Y2)
43	$uminus($uminus(Y0)) = Y0
33	$less($sum(Y0,Y1),$sum(Y2,Y1)) | ~$less(Y0,Y2)
31	0 = $sum(Y0,$uminus(Y0))
30	~$less(Y0,$sum(Y1,1)) | ~$less(Y1,Y0)
25	$sum(Y0,0) = Y0
17	$sum(Y0,$uminus(Y0)) = 0
17	$less(Y0,$sum(Y1,1.0)) | $less(Y1,Y0)
14	$less(Y0,$sum(Y1,1/1)) | $less(Y1,Y0)
13	$product(Y0,Y1) = $product(Y1,Y0)
11	$product(Y0,1) = Y0
9	0.0 = $sum(Y0,$uminus(Y0))
9	0/1 = $sum(Y0,$uminus(Y0))
7	$product(Y0,$product(Y1,Y2)) = $product($product(Y0,Y1),Y2)
7	$product(Y0,$sum(Y1,Y2)) = $sum($product(Y0,Y1),$product(Y0,Y2))
6	$sum(Y0,$uminus(Y0)) = 0.0
4	$sum(Y0,0.0) = Y0
4	$sum(Y0,$uminus(Y0)) = 0/1
4	$product(Y0,0) = 0
3	$uminus($sum(Y0,Y1)) = $sum($uminus(Y1),$uminus(Y0))
3	$sum($product(Y0,Y1),$product(Y0,Y2)) = $product(Y0,$sum(Y1,Y2))
2	$sum(Y0,0/1) = Y0
